# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234169690600                   -0.50    8.0
  2   -7.249915759613       -1.80       -1.40    1.0
  3   -7.251247504490       -2.88       -1.94    4.0
  4   -7.251179771337   +   -4.17       -2.05    6.0
  5   -7.251306187349       -3.90       -2.41    4.0
  6   -7.251335699086       -4.53       -2.87    4.0
  7   -7.251338793224       -5.51       -4.09    3.0
  8   -7.251338795356       -8.67       -4.28    2.0
  9   -7.251338798553       -8.50       -5.22    1.0
 10   -7.251338798676       -9.91       -5.26    3.0
 11   -7.251338798699      -10.64       -5.64    2.0
 12   -7.251338798704      -11.31       -6.25    4.0
 13   -7.251338798704      -12.37       -6.41    3.0
 14   -7.251338798704      -13.07       -6.69    1.0
 15   -7.251338798705      -13.35       -7.37    3.0
 16   -7.251338798705      -14.45       -8.05    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05218481874366396
[ Info: Arnoldi iteration step 2: normres = 0.7836367027189514
[ Info: Arnoldi iteration step 3: normres = 0.7144919942048358
[ Info: Arnoldi iteration step 4: normres = 0.31754591036808405
[ Info: Arnoldi iteration step 5: normres = 0.35984072822709634
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.41e-02, 7.35e-02, 2.15e-01, 2.77e-01, 2.09e-02)
[ Info: Arnoldi iteration step 6: normres = 0.45677853177419847
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (9.21e-03, 1.67e-01, 3.99e-01, 8.53e-02, 1.06e-01)
[ Info: Arnoldi iteration step 7: normres = 0.08168170207521729
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.16e-04, 2.56e-02, 1.91e-02, 3.54e-02, 5.83e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11654869887707416
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.10